### Preprocessing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [118]:
import pandas as pd
import numpy as np

In [132]:
train = pd.read_csv('/content/drive/MyDrive/Team Blue NLP/train_cleaned.csv')
test = pd.read_csv('/content/drive/MyDrive/Team Blue NLP/test_cleaned.csv')
train.head()

,Unnamed: 0,id,comment_text,avg_count_caps,comment_cleaned,comment_tokenized,comment_cleaned_spell,comment_cleaned_no_stopwords,comment_cleaned_spell_no_stopwords,comment_cleaned_no_stopwords_lemm,comment_cleaned_spell_no_stopwords_lemm,comment_cleaned_lemm,comment_cleaned_spell_lemm,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,131272,be631fee6a996d52,List of Moroccan Dutch people \n\nPeople shoul...,0.054422,list of moroccan dutch people people should ...,"['List', 'of', 'Moroccan', 'Dutch', 'people', ...",list of moroccan dutch people people should ha...,list moroccan dutch people people wikipedia pa...,list moroccan dutch people people wikipedia pa...,list moroccan dutch people people wikipedia pa...,list moroccan dutch people people wikipedia pa...,list of moroccan dutch people people should ha...,list of moroccan dutch people people should ha...,0,0,0,0,0,0
1,55898,955b143f75755ac3,"Not much better, be careful should someone rep...",0.012658,not much better be careful should someone rep...,"['Not', 'much', 'better', 'be', 'careful', 'sh...",not much better be careful should someone repo...,much better careful someone report ban vandalism,much better careful someone report ban vandalism,much well careful someone report ban vandalism,much well careful someone report ban vandalism,not much good be careful should someone report...,not much good be careful should someone report...,0,0,0,0,0,0
2,86025,e623261850dd2aaa,I blocked the pair of you because you are not ...,0.022857,i blocked the pair of you because you are not ...,"['I', 'blocked', 'the', 'pair', 'of', 'you', '...",i blocked the pair of you because you are not ...,blocked pair write encyclopaedia plenty places...,blocked pair write encyclopaedia plenty places...,block pair write encyclopaedia plenty place pl...,block pair write encyclopaedia plenty place pl...,i block the pair of you because you be not her...,i block the pair of you because you be not her...,0,0,0,0,0,0
3,96930,068e0a8032a4cf5c,Thank You \n\nThanks for the link on the refer...,0.020833,thank you thanks for the link on the referen...,"['Thank', 'You', 'Thanks', 'for', 'the', 'link...",thank you thanks for the link on the reference...,thank thanks link reference desks crux orthodo...,thank thanks link reference desks crux orthodo...,thank thanks link reference desk crux orthodox...,thank thanks link reference desk crux orthodox...,thank you thank for the link on the reference ...,thank you thank for the link on the reference ...,0,0,0,0,0,0
4,82799,dd852a6da913a096,"I know of about 50,000 Bible Scholars who disa...",0.034146,i know of about _number_ bible scholars who di...,"['I', 'know', 'of', 'about', '_NUMBER_', 'Bibl...",i know of about _number_ bible scholars who di...,know _number_ bible scholars disagree yet libe...,know _number_ bible scholars disagree yet libe...,know _number_ bible scholar disagree yet liber...,know _number_ bible scholar disagree yet liber...,i know of about _number_ bible scholar who dis...,i know of about _number_ bible scholar who dis...,0,0,0,0,0,0


### RNN and LSTM

#### Tokenize your texts

In [133]:
train2=train.copy()
test2=test.copy()

train=train[['comment_cleaned_spell','toxic']].rename(columns={'comment_cleaned_spell':'comment_cleaned'})
test=test[['comment_cleaned_spell','toxic']].rename(columns={'comment_cleaned_spell':'comment_cleaned'})


In [134]:
data=pd.concat([train,test],ignore_index=True)

In [122]:
# toxic=data[data['toxic']==1]
# nontoxic=data[data['toxic']==0]

In [123]:
# from sklearn.utils import resample
# nontoxic_downsample = resample(nontoxic,
#              replace=True,
#              n_samples=len(toxic),
#              random_state=42)
# data=pd.concat([toxic,nontoxic_downsample])

In [135]:
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=10000,oov_token="UNKNOWN_TOKEN") # We have taken top 10000 tokens and rest are marked as "UNKNOWN"
tokenizer.fit_on_texts(data['comment_cleaned'])
# tokenizer.fit_on_texts(test['comment_cleaned'])


##### Set Maximum Sequence Length

In [136]:
# now we will try to find what should be the length of each sentence be allowed for each row.
# anything more than that will be cropped.
# Logically the length of longest sentence is taken. But sometimes outliers can increase exponentially so something shorter is taken

In [137]:
# max=0
# for i in data['comment_cleaned']:
#   if len(i.split(" "))>=max:
#     max=len(i.split(" "))
# max

In [138]:
# t=[len(data['comment_cleaned'][i].split(" ")) for i in range(0,len(data['comment_cleaned']))]

In [139]:
# t[0:5]

In [140]:
# t=pd.Series(t)
# t.mean()

In [ ]:
# t.median()

In [ ]:
# import seaborn as sns
# sns.distplot(t)

In [141]:
MAX_SEQUENCE_LENGTH=200

In [142]:
# So tokenizer has method texts_to_sequences which converts your text data into a list of numbers where each number
# represent the index of tokens created by the tokenizer. for eg, 'You' is represented by 7, "And" is shown by 5. 
# so You is replaced by 7 everywhere and And by 5. In this way we encode our documents with integers.
# tokenizer.word_index  # This will show the tokens with their index

def integer_encode_documents(docs,tokenizer):
  return tokenizer.texts_to_sequences(docs)
data['encoded_comment']=integer_encode_documents(data['comment_cleaned'],tokenizer)
# train['encoded_comment']=integer_encode_documents(train['comment_cleaned'],tokenizer)
# test['encoded_comment']=integer_encode_documents(test['comment_cleaned'],tokenizer)

In [143]:
data[['comment_cleaned','encoded_comment']]

,comment_cleaned,encoded_comment
0,list of moroccan dutch people people should ha...,"[164, 6, 1, 2762, 75, 75, 60, 17, 9, 32, 31, 1..."
1,not much better be careful should someone repo...,"[11, 132, 206, 20, 1582, 60, 153, 533, 7, 29, ..."
2,i blocked the pair of you because you are not ...,"[5, 169, 2, 4846, 6, 8, 72, 8, 19, 11, 65, 3, ..."
3,thank you thanks for the link on the reference...,"[129, 8, 94, 14, 2, 163, 16, 2, 266, 1, 3, 1, ..."
4,i know of about _number_ bible scholars who di...,"[5, 73, 6, 42, 18, 1905, 1836, 66, 726, 318, 2..."
...,...,...
159566,my edit is an addition why would i modify the ...,"[33, 76, 4, 34, 744, 78, 44, 5, 3608, 2, 394, ..."
159567,do not copy text from the bbc into wikipedia t...,"[23, 11, 704, 264, 36, 2, 2240, 141, 32, 15, 4..."
159568,that pdf file does not exist anymore the url i...,"[10, 3888, 490, 77, 11, 773, 1204, 2, 154, 4, ..."
159569,_number_ every vhs is sold in clamshell so thi...,"[18, 288, 1, 4, 2540, 13, 1, 43, 15, 49, 3, 5,..."


In [ ]:
# for i in data.encoded_comment:
#   print(len(i))

In [144]:
# Now length of each document in encoded_comment varies as the length of original document
# So We need to fix the size to MAX_SEQUENCE_LENGTH which in our case is 200

from tensorflow.keras.preprocessing.sequence import pad_sequences

data_x=pad_sequences(data['encoded_comment'],maxlen=MAX_SEQUENCE_LENGTH,padding='post')
# train_x=pad_sequences(train['encoded_comment'],maxlen=MAX_SEQUENCE_LENGTH,padding='post')
# test_x=pad_sequences(test['encoded_comment'],maxlen=MAX_SEQUENCE_LENGTH,padding='post')
# data[['encoded_comment','padded_comment']]

In [145]:
pad_sequences(data['encoded_comment'],maxlen=200,padding='post')

array([[ 164,    6,    1, ...,    0,    0,    0],
       [  11,  132,  206, ...,    0,    0,    0],
       [   5,  169,    2, ...,    0,    0,    0],
       ...,
       [  10, 3888,  490, ...,    0,    0,    0],
       [  18,  288,    1, ...,    0,    0,    0],
       [  50,   23,   11, ...,    0,    0,    0]], dtype=int32)

### Import Keras Toolkit

In [147]:
from random import randint
from numpy import array, argmax, asarray, zeros
from keras.models import Sequential
from keras.layers import Dense, Embedding

In [148]:
VOCAB_SIZE=int(len(tokenizer.word_index)*1.1) # Vocab size is the size of number of unique words we are going to have
                                              # Thus we reserved 10% extra space to all the tokens created. Here its 196060

### Load GloVe Vectors
- These are similar to word2vec. These are loaded from the S3 bucket

In [149]:
!wget https://dso-560-nlp-text-analytics.s3.amazonaws.com/glove6b100dtxt.zip
!unzip glove6b100dtxt.zip

--2022-12-14 05:45:43--  https://dso-560-nlp-text-analytics.s3.amazonaws.com/glove6b100dtxt.zip
Resolving dso-560-nlp-text-analytics.s3.amazonaws.com (dso-560-nlp-text-analytics.s3.amazonaws.com)... 54.231.229.1, 52.217.132.89, 52.217.142.225, ...
Connecting to dso-560-nlp-text-analytics.s3.amazonaws.com (dso-560-nlp-text-analytics.s3.amazonaws.com)|54.231.229.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 137847651 (131M) [application/zip]
Saving to: ‘glove6b100dtxt.zip.1’

glove6b100dtxt.zip. 100%[===================>] 131.46M  77.8MB/s    in 1.7s    

2022-12-14 05:45:45 (77.8 MB/s) - ‘glove6b100dtxt.zip.1’ saved [137847651/137847651]

Archive:  glove6b100dtxt.zip
replace glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: glove.6B.100d.txt       


In [150]:
# Now we will create a function that will return a dictionary, containing words as keys, and 100 dim numpy arrays
# as values.

def load_glove_vectors():
  embeddings={}
  with open('glove.6B.100d.txt') as f:
    for line in f:
      values=line.split()
      word=values[0]
      vector=asarray(values[1:],dtype='float32')
      embeddings[word]=vector
  print("Total vectors loaded in dictionary:",len(embeddings))
  return embeddings



In [151]:
embeddings=load_glove_vectors()
embeddings['bad']

Total vectors loaded in dictionary: 400000


array([ 0.39456  , -0.24717  ,  1.0319   , -0.61444  , -1.2376   ,
        0.051794 , -0.55112  , -0.31014  , -0.025735 , -0.24709  ,
        0.25836  ,  0.34899  ,  0.46176  ,  0.44237  ,  0.354    ,
        0.21262  , -0.26312  , -0.17128  ,  0.44013  ,  1.1284   ,
        0.37259  ,  0.41969  , -0.2534   , -0.051885 , -0.34917  ,
        0.15298  , -0.68444  , -0.19514  , -0.018647 ,  0.084191 ,
        0.14569  ,  0.3869   ,  0.023524 , -0.37861  , -0.65081  ,
        0.010398 , -0.50087  , -0.087368 ,  0.098617 , -0.21907  ,
       -0.072412 , -0.10247  , -0.3148   , -0.307    , -0.30457  ,
       -0.14329  ,  0.10313  , -0.040487 , -0.095922 , -1.192    ,
       -0.063356 ,  0.034127 , -0.057674 ,  0.56582  ,  0.082541 ,
       -2.4617   ,  0.48029  ,  0.72806  ,  1.2841   ,  0.10672  ,
       -0.045119 ,  1.0117   , -0.97928  ,  0.022828 ,  0.46876  ,
       -0.026875 ,  0.80018  , -0.23101  ,  0.086699 , -0.74338  ,
       -0.38738  , -0.1924   , -0.13292  ,  0.52347  ,  0.3565

### Load embeddings for our data

In [152]:
# Now the above embeddings are for all the words trained by glove, i.e for 400,000 words
# Now we will map it to our words list we created using tokenizer
# Our embedding matrix will be vocab size by 100. 
embedding_matrix=zeros((VOCAB_SIZE,100))

for word,i in tokenizer.word_index.items():
  vector1=embeddings.get(word)
  if vector1 is not None:
    embedding_matrix[i]=vector1

In [153]:
embedding_matrix.shape # Now embedding matrix contains word embeddings (size 100 each) for each word (represented by index)
                      # of tokens we generated by tokenizer

(185708, 100)

### Train test split

In [154]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(data_x,data['toxic'].values,test_size=0.2,random_state=104,stratify=data['toxic'])

In [155]:
train_x

array([[1353, 1212, 1081, ...,    0,    0,    0],
       [  75,   10,    1, ..., 7922,    1,    4],
       [1675, 4463,  912, ...,    0,    0,    0],
       ...,
       [   2, 3899,   10, ...,    0,    0,    0],
       [2321,   37, 1683, ...,    0,    0,    0],
       [ 441, 2886,  100, ...,    0,    0,    0]], dtype=int32)

### Define RNN Model

In [157]:
from keras.layers import SimpleRNN, LSTM
from keras.layers import Flatten, Masking
import keras

In [158]:
model=Sequential()
model.add(Embedding(VOCAB_SIZE,100,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False))
model.add(Masking(mask_value=0.0)) # Masking layer, masks any words that don't have an embedding as zero
model.add(SimpleRNN(units=64,input_shape=(1,MAX_SEQUENCE_LENGTH)))
model.add(Dense(16))
model.add(Dense(1,activation='softmax'))

In [159]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[keras.metrics.Precision(),keras.metrics.Recall(),keras.metrics.AUC()])
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 100)          18570800  
                                                                 
 masking_3 (Masking)         (None, 200, 100)          0         
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 64)                10560     
                                                                 
 dense_6 (Dense)             (None, 16)                1040      
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 18,582,417
Trainable params: 11,617
Non-trainable params: 18,570,800
_________________________________________________________________


In [160]:
history=model.fit(train_x,train_y,validation_split=0.2,epochs=2,verbose=1)

Epoch 1/2
3192/3192 [==============================] - 260s 81ms/step - loss: 0.0000e+00 - precision_6: 0.0956 - recall_6: 1.0000 - auc_4: 0.5000 - val_loss: 0.0000e+00 - val_precision_6: 0.0968 - val_recall_6: 1.0000 - val_auc_4: 0.5000
Epoch 2/2
3192/3192 [==============================] - 253s 79ms/step - loss: 0.0000e+00 - precision_6: 0.0956 - recall_6: 1.0000 - auc_4: 0.5000 - val_loss: 0.0000e+00 - val_precision_6: 0.0968 - val_recall_6: 1.0000 - val_auc_4: 0.5000


In [161]:
model.evaluate(test_x,test_y,verbose=1)

998/998 [==============================] - 27s 27ms/step - loss: 0.0000e+00 - precision_6: 0.0958 - recall_6: 1.0000 - auc_4: 0.5000


[0.0, 0.09584834426641464, 1.0, 0.5]

### LSTM

In [162]:
model=Sequential()
model.add(Embedding(VOCAB_SIZE,100,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False))
model.add(Masking(mask_value=0.0)) # Masking layer, masks any words that don't have an embedding as zero
model.add(LSTM(units=32,input_shape=(1,MAX_SEQUENCE_LENGTH)))
model.add(Dense(16))
model.add(Dense(1,activation='softmax'))

In [163]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[keras.metrics.Precision(),keras.metrics.Recall(),keras.metrics.AUC()])
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 200, 100)          18570800  
                                                                 
 masking_4 (Masking)         (None, 200, 100)          0         
                                                                 
 lstm (LSTM)                 (None, 32)                17024     
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 18,588,369
Trainable params: 17,569
Non-trainable params: 18,570,800
_________________________________________________________________


In [164]:
history=model.fit(train_x,train_y,validation_split=0.2,epochs=2,verbose=1)

Epoch 1/2
3192/3192 [==============================] - 396s 123ms/step - loss: 0.0000e+00 - precision_7: 0.0956 - recall_7: 1.0000 - auc_5: 0.5000 - val_loss: 0.0000e+00 - val_precision_7: 0.0968 - val_recall_7: 1.0000 - val_auc_5: 0.5000
Epoch 2/2
3192/3192 [==============================] - 387s 121ms/step - loss: 0.0000e+00 - precision_7: 0.0956 - recall_7: 1.0000 - auc_5: 0.5000 - val_loss: 0.0000e+00 - val_precision_7: 0.0968 - val_recall_7: 1.0000 - val_auc_5: 0.5000


In [165]:
model.evaluate(test_x,test_y,verbose=1)

998/998 [==============================] - 31s 31ms/step - loss: 0.0000e+00 - precision_7: 0.0958 - recall_7: 1.0000 - auc_5: 0.5000


[0.0, 0.09584834426641464, 1.0, 0.5]